In [38]:
import pandas as pd

In [39]:
file_name = "schools_complete.csv"
school_data = pd.read_csv(file_name)

# Remove the rows with missing data
school_data = school_data.dropna(how="any")
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [40]:
file_name = "students_complete.csv"
student_data = pd.read_csv(file_name)

# Remove the rows with missing data
student_data = student_data.dropna(how="any")
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [41]:
total_schools = school_data['School ID'].count()

In [42]:
total_students = student_data['Student ID'].count()

In [43]:
total_budget = school_data.budget.sum()

In [44]:
avg_math_score = round(student_data.math_score.mean(),2)

In [45]:
avg_reading_score = round(student_data.reading_score.mean(),2)

In [46]:
num_passing_math = student_data[student_data.math_score >= 70]['Student ID'].count()
perc_passing_math = round(num_passing_math / total_students * 100,2)

In [47]:
num_passing_reading = student_data[student_data.reading_score >= 70]['Student ID'].count()
perc_passing_reading = round(num_passing_reading / total_students * 100,2)

In [48]:
overall_passing_rate = round((perc_passing_math + perc_passing_reading) / 2,2)

In [49]:
# District Summary

district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [perc_passing_math],
    "% Passing Reading": [perc_passing_reading],
    "Overall Passing Rate": [overall_passing_rate]
})
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.4


In [50]:
merge_table = pd.merge(student_data, school_data, on="school_name")
merge_table.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [51]:
school_groupby = merge_table.groupby('school_name')

In [52]:
school_type = school_groupby.type.max()

In [53]:
total_students_by_school = merge_table['school_name'].value_counts()

In [54]:
total_school_budget = school_groupby.budget.max()

In [55]:
per_student_budget = total_school_budget / total_students_by_school

In [56]:
avg_math_score_by_school = round(school_groupby.math_score.mean(),2)

In [57]:
avg_reading_score_by_school = round(school_groupby.reading_score.mean(),2)

In [58]:
passing_math_filter = merge_table.loc[merge_table.math_score >= 70]
num_passing_math_by_school = passing_math_filter.groupby(["school_name"])["Student ID"].count()
perc_passing_math_by_school = num_passing_math_by_school / total_students_by_school * 100

In [59]:
num_passing_reading_by_school = merge_table.loc[merge_table.reading_score >= 70].groupby(["school_name"])["Student ID"].count()
perc_passing_reading_by_school = num_passing_reading_by_school / total_students_by_school * 100

In [75]:
overall_passing_rate_by_school = (perc_passing_math_by_school + perc_passing_reading_by_school) / 2

In [76]:
overall_summary_by_school_df = pd.DataFrame({
      "School Type": school_type,
      "Total Students": total_students_by_school,
      "Total School Budget": total_school_budget,
      "Per Student Budget": per_student_budget,
      "Average Math Score": avg_math_score_by_school,
      "Average Reading Score": avg_reading_score_by_school,
      "% Passing Math": perc_passing_math_by_school, 
      "% Passing Reading": perc_passing_reading_by_school,
      "Overall Passing Rate": overall_passing_rate_by_school
    })

formatted_summary_df = overall_summary_by_school_df.copy(deep=False)
formatted_summary_df["Total School Budget"] = formatted_summary_df["Total School Budget"].map("${:,.2f}".format)
formatted_summary_df["Total Students"] = formatted_summary_df["Total Students"].map("{:,}".format)
formatted_summary_df["Per Student Budget"] = formatted_summary_df["Per Student Budget"].map("${:,.2f}".format)
formatted_summary_df["% Passing Math"] = round(formatted_summary_df["% Passing Math"],2)
formatted_summary_df["% Passing Reading"] = round(formatted_summary_df["% Passing Reading"],2)
formatted_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.270270


In [77]:
# Top Performing Schools

top5_schools = formatted_summary_df.sort_values("Overall Passing Rate", ascending=False).head()
top5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.203679


In [78]:
# Bottom Performing Schools

bottom5_schools = formatted_summary_df.sort_values("Overall Passing Rate", ascending=False).tail()
bottom5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.804308
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.639992
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.500171
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.363852
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.293323


In [64]:
# Math Scores by Grade

math_scores_9th = merge_table.loc[merge_table.grade=='9th'].groupby(["school_name"]).math_score.mean()
math_scores_10th = merge_table.loc[merge_table.grade=='10th'].groupby(["school_name"]).math_score.mean()
math_scores_11th = merge_table.loc[merge_table.grade=='11th'].groupby(["school_name"]).math_score.mean()
math_scores_12th = merge_table.loc[merge_table.grade=='12th'].groupby(["school_name"]).math_score.mean()

mean_math_scores = pd.DataFrame({
    "9th": math_scores_9th,
    "10th": math_scores_10th,
    "11th": math_scores_11th,
    "12th": math_scores_12th
})
mean_math_scores


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [65]:
# Reading Scores by Grade

reading_scores_9th = merge_table.loc[merge_table.grade=='9th'].groupby(["school_name"]).reading_score.mean()
reading_scores_10th = merge_table.loc[merge_table.grade=='10th'].groupby(["school_name"]).reading_score.mean()
reading_scores_11th = merge_table.loc[merge_table.grade=='11th'].groupby(["school_name"]).reading_score.mean()
reading_scores_12th = merge_table.loc[merge_table.grade=='12th'].groupby(["school_name"]).reading_score.mean()

mean_reading_scores = pd.DataFrame({
    "9th": reading_scores_9th,
    "10th": reading_scores_10th,
    "11th": reading_scores_11th,
    "12th": reading_scores_12th
})
mean_reading_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [66]:
bins_list = [0, 585, 615, 645, 675]
labels_list = ["<$585", "$585-$615", "$615-$645", "$645-$675"]
overall_summary_by_school_df['Spending Range'] = pd.cut(overall_summary_by_school_df['Per Student Budget'], bins=bins_list, labels=labels_list)

In [67]:
avg_math_score = overall_summary_by_school_df.groupby('Spending Range')['Average Math Score'].mean()
avg_reading_score = overall_summary_by_school_df.groupby('Spending Range')['Average Reading Score'].mean()
perc_passing_math = overall_summary_by_school_df.groupby('Spending Range')['% Passing Math'].mean()
perc_passing_reading = overall_summary_by_school_df.groupby('Spending Range')['% Passing Reading'].mean()
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [68]:
scores_by_spending_df = pd.DataFrame ({
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passing Math": perc_passing_math,
    "% Passing Reading": perc_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})
scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Range,,,,,
<$585,83.452500,83.935000,93.460096,96.610877,95.035486
$585-$615,83.600000,83.885000,94.230858,95.900287,95.065572
$615-$645,79.078333,81.891667,75.668212,86.106569,80.887391
$645-$675,76.996667,81.026667,66.164813,81.133951,73.649382


In [69]:
bins_list = [0, 1000, 2000, 5000]
labels_list = ["Small (< 1000)", "Medium (1000 - 2000)", "Large (2000 - 5000)"]
overall_summary_by_school_df['School Size'] = pd.cut(overall_summary_by_school_df['Total Students'], bins=bins_list, labels=labels_list)

In [70]:
avg_math_score = overall_summary_by_school_df.groupby('School Size')['Average Math Score'].mean()
avg_reading_score = overall_summary_by_school_df.groupby('School Size')['Average Reading Score'].mean()
perc_passing_math = overall_summary_by_school_df.groupby('School Size')['% Passing Math'].mean()
perc_passing_reading = overall_summary_by_school_df.groupby('School Size')['% Passing Reading'].mean()
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [71]:
scores_by_school_size_df = pd.DataFrame ({
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passign Math": perc_passing_math,
    "% Passign Reading": perc_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})
scores_by_school_size_df

,Average Math Score,Average Reading Score,% Passign Math,% Passign Reading,Overall Passing Rate
School Size,,,,,
Small (< 1000),83.820,83.92500,93.550225,96.099437,94.824831
Medium (1000 - 2000),83.374,83.86800,93.599695,96.790680,95.195187
Large (2000 - 5000),77.745,81.34375,69.963361,82.766634,76.364998


In [72]:
school_type_groupby = overall_summary_by_school_df.groupby('School Type')

In [73]:
avg_math_score = school_type_groupby['Average Math Score'].mean()
avg_reading_score = school_type_groupby['Average Reading Score'].mean()
perc_passing_math = school_type_groupby['% Passing Math'].mean()
perc_passing_reading = school_type_groupby['% Passing Reading'].mean()
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [74]:
scores_by_school_type_df = pd.DataFrame ({
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passing Math": perc_passing_math,
    "% Passing Reading": perc_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})
scores_by_school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.472500,83.897500,93.620830,96.586489,95.103660
District,76.955714,80.965714,66.548453,80.799062,73.673757
